### Création du tableau de comparaison des marchés et tunnels

On essait de créer un fichier .csv qui importe des données provenant de deux sources principales : le fichier 'prodQuantAnLieuUniType2324.csv' pour les actions préventives en 2023 et 2024, et le fichier 'Tunnels_Cout préventif_05.05.2025.xlsx' qui contient les actions préventives pour 2025. 

Enfin, on veut illustrer les lieux, les marchés et les montants pour les années 2023-2025.

##### Marchés 2023/24

In [1]:
import pandas as pd

# Importer les données des préventives pour 2023 et 2024
prodQuantAnLieuUniTyp2324 = pd.read_csv('https://storage.googleapis.com/sucombe-dirif/reference/prodQuantAnLieuUniTyp2324.csv')


# Filtrer uniquement les lignes de type préventif
preventifs = prodQuantAnLieuUniTyp2324[prodQuantAnLieuUniTyp2324['typeP'] == 'Preventif'] 

# Retirer les lieux non pertinents de l'analyse
lieux_exclus = ['PCTT', 'SIRIUS', 'SIREDO']
preventifs = preventifs[~preventifs['lieu'].isin(lieux_exclus)]

# Supprimer les colonnes inutiles
preventifs_filtre = preventifs.drop(columns=[
    'com_prod_quantite', 'quantite_constate', 'unite', 'prod_id'
])

# Regrouper par lieu, marché, année et sommer les montants
preventif_groupe = preventifs_filtre.groupby(
    ['lieu', 'mar_diminutif', 'com_bud_annee']
)['com_prod_montant_ht'].sum().reset_index()

# Transformer les années en colonnes séparées (2023 et 2024)
preventif_pivote = preventif_groupe.pivot_table(
    index=['lieu', 'mar_diminutif'],
    columns='com_bud_annee',
    values='com_prod_montant_ht',
    fill_value=0
).reset_index()

# Supprimer le nom de l’index des colonnes et renommer les colonnes pivotées
preventif_pivote.columns.name = None
preventif_pivote = preventif_pivote.rename(columns={2023: 'montant_2023', 2024: 'montant_2024'})

# Regrouper certains tunnels sous un même nom
lieu_map = {
    'Chennevières': "Chennevières", 'Antony': "Antony", 'Bicêtre': 'Bicêtre', 'Landy': 'Landy',
    'Nanterre / La Défense': 'Nanterre / La Défense', 'Nogent': 'Nogent',
    'Saint Cloud': 'Ambroise Paré + Saint-Cloud', 'Taverny': 'Taverny',
    'Boissy-Saint-Léger': 'Boissy', 'Bobigny-Lumen-Norton': 'Bobigny-Lumen-Norton',
    'La Courneuve': 'La Courneuve', 'Thiais': 'Thiais', 'Orly': 'Orly', 'Neuilly': 'Neuilly',
    'Ambroise PARE': 'Ambroise Paré + Saint-Cloud', 'Bellerive': 'Bellerive', 'Fresnes': 'Fresnes',
    'Italie': 'Italie', 'Champigny': 'Champigny', 'Sévines': 'Sévines',
    'Fontenay le Fleury': 'Fontenay Le Fleury', 'Tous tunnels': 'Divers', 'SIRIUS et TUNNELS': 'Divers'
}
preventif_pivote['lieu'] = preventif_pivote['lieu'].map(lieu_map)

# Harmoniser les noms des marchés
marche_map = {
    'AEV': 'AEV', 'Automates': 'Automate', 'Automates23': 'Automate', 'Bat': 'Bâtiment',
    'Cont Reg': 'ContReg', 'Eclairage': 'Eclairage', 'MEC2': 'MEC', 'Onduleur': 'Onduleur',
    'Onduleur2': 'Onduleur', 'Pompage': 'Pompage', 'Pompage 2': 'Pompage',
    'Propreté': 'Propreté', 'Ventilation2': 'Ventilation', 'Ventilation3': 'Ventilation',
    'Vidéo': 'Vidéo', 'Vidéo 2': 'Vidéo',
    'RAU/TSE': 'PAU/TSE', 'RAU/TSE23': 'PAU/TSE',
    'MEC3-Lot1': 'MEC', 'Detection2': 'Détection', 'Detection3': 'Détection'
}
preventif_pivote['mar_diminutif'] = preventif_pivote['mar_diminutif'].map(marche_map).fillna(preventif_pivote['mar_diminutif'])

# Classer certains marchés comme "Divers"
divers_marches = [
    'CS', 'MDIRIF', 'SignaDyn', 'AMO Tunnel', 'CLIM2',
    'Climatisation2', 'CAOA', 'MIISST2'
]

preventif_pivote['mar_diminutif'] = preventif_pivote.apply(
    lambda row: 'Divers' if row['mar_diminutif'] in divers_marches else f"{row['mar_diminutif']}",
    axis=1
)

# Regrouper les données finales par lieu et marché, avec montants 2023 et 2024
final_df_23_24 = preventif_pivote.groupby(
    ['lieu', 'mar_diminutif'], as_index=False
)[['montant_2023', 'montant_2024']].sum()

##### Marchés 2025

In [2]:
import pandas as pd
import requests
from io import BytesIO
from openpyxl import load_workbook
import numpy as np

# Config : chemin du fichier et liste des tunnels à traiter
file_path = 'https://storage.googleapis.com/sucombe-dirif/reference/Tunnels_Cout%20pr%C3%A9ventif_04.04.2024.xlsx'
tunnels = [
    'Boissy', 'Champigny', 'Guy Môquet', 'Moulin', 'Nogent',
    'Ambroise Paré', 'Belle-Rive', 'Chennevières', 'Fontenay', 'La Défense',
    'Nanterre Centre', 'Nanterre échangeur', 'Neuilly', 'Saint-Cloud', 'Sévines',
    'Bobigny', 'La Courneuve', 'Landy', 'Lumen-Norton', 'Taverny',
    'Antony', 'Fresnes', 'Bicêtre', 'Italie', 'Orly',
]

# Télécharger le fichier depuis l'URL
response = requests.get(file_path)
response.raise_for_status()  # Lève une erreur si le téléchargement échoue

# Charger le fichier Excel 
wb = load_workbook(filename=BytesIO(response.content), data_only=True)
rows = []

# Parcourir chaque feuille correspondant à un tunnel et extraire les montants 2025
for sheet in tunnels:
    ws = wb[sheet]
    marche_montants = {}

    for row in ws.iter_rows(min_row=3, max_col=8):  # Colonnes A à H
        marche = row[0].value  # Colonne A : marché
        montant = row[7].value  # Colonne H : montant 2025

        if marche and isinstance(montant, (int, float)):
            marche_montants[marche] = marche_montants.get(marche, 0.0) + montant

    for marche, total in marche_montants.items():
        rows.append({
            'Tunnel': sheet,
            'Marché': marche,
            'Montant 2025 HT': total
        })

# Créer un DataFrame à partir des données extraites
mar_tun_25 = pd.DataFrame(rows)

# Trier les données par tunnel et marché
mar_tun_25 = mar_tun_25.sort_values(by=['Tunnel', 'Marché']).reset_index(drop=True)

# Regrouper certains tunnels sous un même nom (cohérence avec 2023-2024)
tunnel_map = {
    'Ambroise Paré': 'Ambroise Paré + Saint-Cloud',
    'Saint-Cloud': 'Ambroise Paré + Saint-Cloud',
    'Guy Môquet': 'Thiais',
    'Moulin': 'Thiais',
    'Belle-Rive': 'Bellerive',
    'Fontenay': 'Fontenay Le Fleury',
    'La Défense': 'Nanterre / La Défense',
    'Nanterre Centre': 'Nanterre / La Défense',
    'Nanterre échangeur': 'Nanterre / La Défense',
    'Bobigny': 'Bobigny-Lumen-Norton',
    'Lumen-Norton': 'Bobigny-Lumen-Norton'
}

# Appliquer la correspondance sur les noms de tunnel
mar_tun_25['Tunnel Group'] = mar_tun_25['Tunnel'].map(tunnel_map).fillna(mar_tun_25['Tunnel'])

# Regrouper par tunnel et marché, puis additionner les montants
grouped = (
    mar_tun_25
    .groupby(['Tunnel Group', 'Marché'], as_index=False)['Montant 2025 HT']
    .sum()
)

# Renommer les colonnes pour harmoniser avec les autres DataFrames
grouped = grouped.rename(columns={
    'Tunnel Group': 'Tunnel',
    'Marché': 'Marché',
    'Montant 2025 HT': 'Montant 2025 HT'
})

# DataFrame final pour 2025
mar_tun_25_grouped = grouped.sort_values(by=['Tunnel', 'Marché']).reset_index(drop=True)

# Fonction d’arrondi à 2 chiffres significatifs
def round_sig(x, sig=2):
    if isinstance(x, (int, float)) and x != 0:
        return f"{int(round(x, -int(np.floor(np.log10(abs(x))) - (sig - 1))))}"
    elif x == 0:
        return "0"
    else:
        return x

# Renommer les colonnes pour correspondre à celles du DataFrame 2023-2024
mar_tun_25_grouped = mar_tun_25_grouped.rename(columns={
    'Tunnel': 'lieu',
    'Marché': 'mar_diminutif'
})

# Arrondir les montants à 2 chiffres significatifs
mar_tun_25_grouped['Montant 2025 HT'] = mar_tun_25_grouped['Montant 2025 HT'].apply(round_sig)

# Fusionner les données 2025 avec celles de 2023-2024
final_df = final_df_23_24.merge(
    mar_tun_25_grouped,
    on=['lieu', 'mar_diminutif'],
    how='left'
)

# Masquer les montants 2025 pour les marchés classés "Divers"
final_df['Montant 2025 HT'] = final_df.apply(
    lambda row: '-' if row['mar_diminutif'] == 'Divers' else row['Montant 2025 HT'],
    axis=1
)

# Arrondir les colonnes numériques à 2 chiffres significatifs
for col in final_df.select_dtypes(include=['int64', 'float64']).columns:
    final_df[col] = final_df[col].apply(round_sig)

# Réorganiser les colonnes pour mettre "Montant 2025 HT" à la fin
cols = [col for col in final_df.columns if col != 'Montant 2025 HT'] + ['Montant 2025 HT']
final_df = final_df[cols]


#### Intégrer les montants 2025 des marchés "Tous tunnels" dans les lignes "Divers"

# Charger les données depuis la feuille "Préventifs_tunnels"
df_preventif = pd.read_excel(file_path, sheet_name='Préventifs_tunnels', usecols='B,F,G,P')
df_preventif.columns = ['mar_diminutif', 'Type', 'Unité', 'Fréquence']

# Filtrer uniquement les lignes de type "Tous tunnels"
df_tous_tunnels = df_preventif[df_preventif['Type'] == 'Tous tunnels'].copy()

# Calculer les montants (Unité x Fréquence)
df_tous_tunnels['Montant 2025 HT'] = df_tous_tunnels['Unité'] * df_tous_tunnels['Fréquence']
df_tous_tunnels = df_tous_tunnels[['mar_diminutif', 'Montant 2025 HT']]
df_tous_tunnels['Montant 2025 HT'] = df_tous_tunnels['Montant 2025 HT'].apply(round_sig)

# Séparer les lignes "Divers" du reste du DataFrame
final_df_non_divers = final_df[final_df['lieu'] != 'Divers'].copy()
final_df_divers = final_df[final_df['lieu'] == 'Divers'].copy()

# Fusionner les montants 2025 avec les lignes "Divers"
updated_divers = final_df_divers.merge(df_tous_tunnels, on='mar_diminutif', how='outer', suffixes=('', '_new'))

# Remplacer les montants vides avec les nouvelles valeurs calculées
updated_divers['Montant 2025 HT'] = updated_divers['Montant 2025 HT_new'].combine_first(updated_divers['Montant 2025 HT'])

# Nettoyer les colonnes temporaires
updated_divers.drop(columns=['Montant 2025 HT_new'], inplace=True)
updated_divers['lieu'] = 'Divers'  # S’assurer que le champ "lieu" reste "Divers"

# Recomposer le DataFrame complet avec les lignes "Divers" mises à jour
final_df = pd.concat([final_df_non_divers, updated_divers], ignore_index=True)

# Garder une seule ligne par marché pour "Divers", en conservant la valeur la plus élevée
divers_df = final_df[final_df['lieu'] == 'Divers']
divers_df = (
    divers_df.sort_values(by='Montant 2025 HT', ascending=False)
    .drop_duplicates(subset='mar_diminutif', keep='first')
)

non_divers_df = final_df[final_df['lieu'] != 'Divers']
final_df = pd.concat([non_divers_df, divers_df], ignore_index=True)

# Remplacer les valeurs manquantes restantes par '-'
final_df = final_df.fillna('-')


##### Tableau Final

Maintenant on peut enregistrer final_df dans un fichier csv.

In [3]:
# final_df.to_csv('preventifs_23_24_25.csv')

### Traiter le tableau 

Au début, on veut trouver les écarts les plus significatifs entre les montants constatés en 2023/24 et les valeurs normatives pour 2025.

##### Marchés (écarts positifs et négatifs)

In [4]:
import pandas as pd
import numpy as np


# Charger le fichier CSV contenant les données 2023, 2024, 2025
df = pd.read_csv("preventifs_23_24_25.csv")


# Convertir les colonnes de montants en types numériques (gestion des erreurs)
df["montant_2023"] = pd.to_numeric(df["montant_2023"], errors="coerce")
df["montant_2024"] = pd.to_numeric(df["montant_2024"], errors="coerce")
df["Montant 2025 HT"] = pd.to_numeric(df["Montant 2025 HT"], errors="coerce")


# Regrouper les données par marché et sommer les montants pour chaque année
grouped = df.groupby("mar_diminutif")[["montant_2023", "montant_2024", "Montant 2025 HT"]].sum()


# Calculer la moyenne des montants de 2023 et 2024
grouped["moyenne_2023_2024"] = grouped[["montant_2023", "montant_2024"]].mean(axis=1)


# Calculer la différence relative en pourcentage entre la moyenne 23/24 et 2025
grouped["relative_difference_%"] = ((grouped["moyenne_2023_2024"] - grouped["Montant 2025 HT"]) / grouped["moyenne_2023_2024"]) * 100


# Calculer la différence absolue entre la moyenne 23/24 et 2025
grouped["absolute_difference"] = grouped["moyenne_2023_2024"] - grouped["Montant 2025 HT"]


# Supprimer les colonnes d’origine de 2023 et 2024
grouped = grouped.drop(columns=["montant_2023", "montant_2024"])


# Arrondir les valeurs pour améliorer la lisibilité
grouped = grouped.round(2)


# Trier par différence absolue décroissante (pour repérer les plus grands écarts)
grouped.sort_values(by="absolute_difference", axis=0, ascending=False)


# Calculer le total de la moyenne 23/24 pour tous les marchés
total_moyenne = grouped["moyenne_2023_2024"].sum()


# Ajouter une colonne représentant la part de chaque marché dans le total
grouped["part_du_total (%)"] = (grouped["moyenne_2023_2024"] / total_moyenne * 100).round(2)


# Réorganiser les colonnes pour mettre "% du total" en premier
cols = ["part_du_total (%)"] + [col for col in grouped.columns if col != "part_du_total (%)"]
grouped = grouped[cols]


# Fonction pour arrondir à deux chiffres significatifs
def round_sf(x, sig=2):
    if pd.isnull(x) or x == 0:
        return x
    return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)


# Appliquer cet arrondi à toutes les colonnes numériques
numeric_cols = grouped.select_dtypes(include=[np.number]).columns
grouped[numeric_cols] = grouped[numeric_cols].applymap(round_sf)


# Convertir les valeurs en kilo-euros
grouped["Montant 2025 HT"] = grouped["Montant 2025 HT"] / 1000
grouped["moyenne_2023_2024"] = grouped["moyenne_2023_2024"] / 1000
grouped["absolute_difference"] = grouped["absolute_difference"] / 1000


# Renommer les colonnes pour indiquer les unités (k€)
grouped.rename(columns={
    "Montant 2025 HT": "Montant 2025 HT (k€)",
    "moyenne_2023_2024": "Moyenne 2023/24 HT (k€)",
    "part_du_total (%)": "% du total 23/24",
    "relative_difference_%": "Différence Relative (%)",
    "absolute_difference": "Différence Absolue k€"
}, inplace=True)


# Supprimer la ligne "Divers", qui n’est pas pertinente pour cette analyse
grouped = grouped.drop("Divers", axis=0)

# Afficher le résultat final
grouped


,% du total 23/24,Montant 2025 HT (k€),Moyenne 2023/24 HT (k€),Différence Relative (%),Différence Absolue k€
mar_diminutif,,,,,
AEV,3.00,500.0,150.0,-230.00,-350.0
Automate,1.30,60.0,65.0,8.40,5.5
Bâtiment,2.20,810.0,110.0,-620.00,-700.0
ContReg,7.90,270.0,400.0,33.00,130.0
Détection,12.00,750.0,590.0,-28.00,-160.0
Eclairage,4.50,940.0,230.0,-320.00,-720.0
MEC,14.00,550.0,690.0,20.00,140.0
Onduleur,0.62,24.0,31.0,23.00,7.2
PAU/TSE,1.70,32.0,86.0,63.00,54.0


Maintenant, on veut exploiter les 5 marchés avec les plus écarts significatifs (positifs et négatifs)



In [5]:
top5 = grouped.sort_values(by="Différence Absolue k€", ascending=False).head(5)
bottom5 = grouped.sort_values(by="Différence Absolue k€").head(5)


In [6]:
top5

,% du total 23/24,Montant 2025 HT (k€),Moyenne 2023/24 HT (k€),Différence Relative (%),Différence Absolue k€
mar_diminutif,,,,,
MEC,14.00,550.0,690.0,20.0,140.0
ContReg,7.90,270.0,400.0,33.0,130.0
Pompage,4.90,160.0,250.0,36.0,90.0
PAU/TSE,1.70,32.0,86.0,63.0,54.0
Onduleur,0.62,24.0,31.0,23.0,7.2


In [7]:
bottom5

,% du total 23/24,Montant 2025 HT (k€),Moyenne 2023/24 HT (k€),Différence Relative (%),Différence Absolue k€
mar_diminutif,,,,,
Eclairage,4.5,940.0,230.0,-320.0,-720.0
Bâtiment,2.2,810.0,110.0,-620.0,-700.0
AEV,3.0,500.0,150.0,-230.0,-350.0
Détection,12.0,750.0,590.0,-28.0,-160.0
Propreté,16.0,950.0,800.0,-19.0,-150.0


##### Trouver les tunnels qui constituent le % le plus grande

Question: quelles sont les tunnels qui consitutent la majorité des écarts dans les marchés?

In [8]:
# Définir l’ordre souhaité des 5 marchés principaux
ordered_top5 = ["MEC", "ContReg", "Pompage", "PAU/TSE", "Onduleur"]


# Filtrer le DataFrame pour ne conserver que les 5 marchés sélectionnés
filtered_df = df[df["mar_diminutif"].isin(ordered_top5)].copy()


# Calculer la moyenne des montants 2023 et 2024 pour chaque tunnel
filtered_df["avg_2023_2024"] = filtered_df[["montant_2023", "montant_2024"]].mean(axis=1)


# Calculer le total des montants 2025 et la moyenne 2023/24 par marché 
total_2025_by_marche = filtered_df.groupby("mar_diminutif")["Montant 2025 HT"].sum()
total_avg_by_marche = filtered_df.groupby("mar_diminutif")["avg_2023_2024"].sum()


# Reprojeter les totaux par marché sur chaque ligne du DataFrame
filtered_df["total_2025_for_marche"] = filtered_df["mar_diminutif"].map(total_2025_by_marche)
filtered_df["total_avg_for_marche"] = filtered_df["mar_diminutif"].map(total_avg_by_marche)


# Calculer le pourcentage de chaque tunnel dans son marché (2025 et moyenne 23/24)
filtered_df["%_2025_du_marche"] = (filtered_df["Montant 2025 HT"] / filtered_df["total_2025_for_marche"]) * 100
filtered_df["%_avg_du_marche"] = (filtered_df["avg_2023_2024"] / filtered_df["total_avg_for_marche"]) * 100


# Sélectionner les 3 tunnels les plus coûteux (en 2025) pour chaque marché
top_tunnels_by_marche = (
    filtered_df.sort_values(["mar_diminutif", "Montant 2025 HT"], ascending=[True, False])
    .groupby("mar_diminutif")
    .head(3)
)


# Sélectionner et renommer les colonnes pertinentes pour le résumé final
top_tunnels_summary = top_tunnels_by_marche[[
    "mar_diminutif", "lieu", "Montant 2025 HT", "%_2025_du_marche",
    "avg_2023_2024", "%_avg_du_marche"
]].rename(columns={
    "mar_diminutif": "Marché",
    "lieu": "Tunnel",
    "Montant 2025 HT": "2025 Cost (€)",
    "%_2025_du_marche": "% of 2025 Total",
    "avg_2023_2024": "Avg 23/24 (€)",
    "%_avg_du_marche": "% of Avg Total"
})


# Définir l’ordre des marchés comme une catégorie ordonnée
top_tunnels_summary["Marché"] = pd.Categorical(top_tunnels_summary["Marché"], categories=ordered_top5, ordered=True)


# Trier les résultats selon l’ordre défini et le coût décroissant de 2025
top_tunnels_summary = top_tunnels_summary.sort_values(["Marché", "2025 Cost (€)"], ascending=[True, False])


# Arrondir les résultats pour une meilleure lisibilité
top_tunnels_summary = top_tunnels_summary.round(2)


# Convertir les valeurs monétaires en kilo-euros
top_tunnels_summary["2025 Cost (k€)"] = top_tunnels_summary["2025 Cost (€)"] / 1000
top_tunnels_summary["Avg 23/24 (k€)"] = top_tunnels_summary["Avg 23/24 (€)"] / 1000


# Supprimer les colonnes initiales en euros
top_tunnels_summary.drop(columns=["2025 Cost (€)", "Avg 23/24 (€)"], inplace=True)


# Définir une fonction pour arrondir à 2 chiffres significatifs
def round_sf(x, sig=2):
    if pd.isnull(x) or x == 0:
        return x
    return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)


# Appliquer l’arrondi significatif aux colonnes numériques
numeric_cols = top_tunnels_summary.select_dtypes(include=[np.number]).columns
top_tunnels_summary[numeric_cols] = top_tunnels_summary[numeric_cols].applymap(round_sf)


# Réorganiser les colonnes (optionnel)
top_tunnels_summary = top_tunnels_summary[[
    "Marché", "Tunnel", "2025 Cost (k€)", "% of 2025 Total",
    "Avg 23/24 (k€)", "% of Avg Total"
]]


# Remplacer les éventuelles valeurs manquantes par un tiret pour l'affichage
top_tunnels_summary = top_tunnels_summary.fillna('-')


# Afficher le tableau final
top_tunnels_summary


,Marché,Tunnel,2025 Cost (k€),% of 2025 Total,Avg 23/24 (k€),% of Avg Total
159,MEC,Nanterre / La Défense,190.0,35.0,180.0,26.0
59,MEC,Bobigny-Lumen-Norton,48.0,8.8,88.0,13.0
246,MEC,Divers,47.0,8.6,-,-
156,ContReg,Nanterre / La Défense,170.0,64.0,160.0,40.0
3,ContReg,Ambroise Paré + Saint-Cloud,18.0,6.7,16.0,4.2
181,ContReg,Nogent,16.0,6.0,30.0,7.7
162,Pompage,Nanterre / La Défense,52.0,33.0,87.0,35.0
22,Pompage,Antony,25.0,16.0,28.0,11.0
35,Pompage,Bellerive,15.0,9.6,25.0,10.0
161,PAU/TSE,Nanterre / La Défense,12.0,37.0,38.0,45.0


In [9]:
# Définir l’ordre souhaité des 5 marchés 
ordered_bottom5 = ["Eclairage", "Bâtiment", "AEV", "Détection", "Propreté"]


# Filtrer le DataFrame pour ne conserver que les 5 marchés sélectionnés
filtered_df = df[df["mar_diminutif"].isin(ordered_bottom5)].copy()


# Calculer la moyenne des montants 2023 et 2024 pour chaque tunnel
filtered_df["avg_2023_2024"] = filtered_df[["montant_2023", "montant_2024"]].mean(axis=1)


# Calculer les totaux des montants 2025 et les moyennes 2023/24 par marché
total_2025_by_marche = filtered_df.groupby("mar_diminutif")["Montant 2025 HT"].sum()
total_avg_by_marche = filtered_df.groupby("mar_diminutif")["avg_2023_2024"].sum()


# Reprojeter les totaux par marché sur chaque ligne du DataFrame
filtered_df["total_2025_for_marche"] = filtered_df["mar_diminutif"].map(total_2025_by_marche)
filtered_df["total_avg_for_marche"] = filtered_df["mar_diminutif"].map(total_avg_by_marche)


# Calculer le pourcentage de chaque tunnel dans son marché (pour 2025 et pour la moyenne 23/24)
filtered_df["%_2025_du_marche"] = (filtered_df["Montant 2025 HT"] / filtered_df["total_2025_for_marche"]) * 100
filtered_df["%_avg_du_marche"] = (filtered_df["avg_2023_2024"] / filtered_df["total_avg_for_marche"]) * 100


# Sélectionner les 3 tunnels les plus coûteux (en 2025) pour chaque marché
bottom_tunnels_by_marche = (
    filtered_df.sort_values(["mar_diminutif", "Montant 2025 HT"], ascending=[True, False])
    .groupby("mar_diminutif")
    .head(3)
)


# Sélectionner et renommer les colonnes pertinentes pour le résumé final
bottom_tunnels_summary = bottom_tunnels_by_marche[[
    "mar_diminutif", "lieu", "Montant 2025 HT", "%_2025_du_marche",
    "avg_2023_2024", "%_avg_du_marche"
]].rename(columns={
    "mar_diminutif": "Marché",
    "lieu": "Tunnel",
    "Montant 2025 HT": "2025 Cost (€)",
    "%_2025_du_marche": "% of 2025 Total",
    "avg_2023_2024": "Avg 23/24 (€)",
    "%_avg_du_marche": "% of Avg Total"
})


# Définir l’ordre des marchés comme une catégorie ordonnée
bottom_tunnels_summary["Marché"] = pd.Categorical(bottom_tunnels_summary["Marché"], categories=ordered_bottom5, ordered=True)


# Trier les résultats selon l’ordre défini et le coût décroissant de 2025
bottom_tunnels_summary = bottom_tunnels_summary.sort_values(["Marché", "2025 Cost (€)"], ascending=[True, False])


# Arrondir les résultats pour une meilleure lisibilité
bottom_tunnels_summary = bottom_tunnels_summary.round(2)


# Convertir les valeurs monétaires en kilo-euros
bottom_tunnels_summary["2025 Cost (k€)"] = bottom_tunnels_summary["2025 Cost (€)"] / 1000
bottom_tunnels_summary["Avg 23/24 (k€)"] = bottom_tunnels_summary["Avg 23/24 (€)"] / 1000


# Supprimer les colonnes initiales en euros
bottom_tunnels_summary.drop(columns=["2025 Cost (€)", "Avg 23/24 (€)"], inplace=True)


# Définir une fonction pour arrondir à 2 chiffres significatifs
def round_sf(x, sig=2):
    if pd.isnull(x) or x == 0:
        return x
    return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)


# Appliquer l’arrondi significatif aux colonnes numériques
numeric_cols = bottom_tunnels_summary.select_dtypes(include=[np.number]).columns
bottom_tunnels_summary[numeric_cols] = bottom_tunnels_summary[numeric_cols].applymap(round_sf)


# Réorganiser les colonnes (optionnel)
bottom_tunnels_summary = bottom_tunnels_summary[[
    "Marché", "Tunnel", "2025 Cost (k€)", "% of 2025 Total",
    "Avg 23/24 (k€)", "% of Avg Total"
]]


# Remplacer les éventuelles valeurs manquantes par un tiret pour l'affichage
bottom_tunnels_summary = bottom_tunnels_summary.fillna('-')


# Affichage du tableau final
bottom_tunnels_summary


,Marché,Tunnel,2025 Cost (k€),% of 2025 Total,Avg 23/24 (k€),% of Avg Total
184,Eclairage,Nogent,160.0,17.0,42.0,18.0
234,Eclairage,Thiais,130.0,14.0,30.0,13.0
58,Eclairage,Bobigny-Lumen-Norton,100.0,11.0,20.0,8.8
2,Bâtiment,Ambroise Paré + Saint-Cloud,130.0,16.0,10.0,9.4
155,Bâtiment,Nanterre / La Défense,110.0,14.0,16.0,15.0
28,Bâtiment,Bellerive,80.0,9.9,3.1,2.8
153,AEV,Nanterre / La Défense,170.0,34.0,41.0,27.0
65,AEV,Boissy,100.0,20.0,3.0,2.0
52,AEV,Bobigny-Lumen-Norton,30.0,6.0,16.0,10.0
158,Détection,Nanterre / La Défense,140.0,19.0,140.0,25.0


##### Marchés Divers

Enfin, on va essayer de exploiter les données sur les Marchés Divers. Comme déjà montré dans le tableau de correspondance, les marchés divers sont: 'CS', 'MDIRIF', 'SignaDyn', 'AMO Tunnel', 'CLIM2',
'Climatisation2', 'CAOA', et 'MIISST2'.

In [10]:
# Fonction pour arrondir à 2 chiffres significatifs
def round_sf(x, sig=2):
    if pd.isnull(x) or x == 0:
        return x
    return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)


# Charger les données
df = pd.read_csv('https://storage.googleapis.com/sucombe-dirif/reference/prodQuantAnLieuUniTyp2324.csv')



# Listes de marchés et lieux à filtrer
selected_marches = ['CS', 'MDIRIF', 'SignaDyn', 'AMO Tunnel', 'CLIM2',
                    'Climatisation2', 'CAOA', 'MIISST2']


all_marches = ['AEV', 'AMO Tunnel', 'Automates', 'Automates23', 'Bat', 'CAOA',
               'CLIM2', 'CS', 'Climatisation2', 'Cont Reg', 'Detection2',
               'Detection3', 'Eclairage', 'MDIRIF', 'MEC2',
               'MEC3-Lot1', 'MEC3-Lot2', 'MIISST2', 'Onduleur', 'Onduleur2',
               'Pompage', 'Pompage 2', 'Propreté', 'RAU/TSE', 'RAU/TSE23',
               'SignaDyn', 'Ventilation2', 'Ventilation3', 'Vidéo', 'Vidéo 2']


lieux = ['Chennevières', 'Antony', 'Bicêtre', 'Landy',
         'Nanterre / La Défense', 'Nogent', 'Saint Cloud',
         'Taverny', 'Boissy-Saint-Léger', 'Bobigny-Lumen-Norton',
         'La Courneuve', 'Thiais', 'Orly', 'Neuilly', 'Ambroise PARE',
         'Bellerive', 'Fresnes', 'Italie', 'Champigny', 'Sévines',
         'Fontenay le Fleury', 'Tous tunnels', 'SIRIUS']


# Filtrage des lignes pertinentes
df_filtered = df[
    (df["typeP"] == "Preventif") &
    (df["lieu"].isin(lieux)) &
    (df["mar_diminutif"].isin(all_marches)) &
    (df["com_bud_annee"].isin([2023, 2024]))
].copy()


# Conversion en numérique
df_filtered["com_prod_montant_ht"] = pd.to_numeric(df_filtered["com_prod_montant_ht"], errors="coerce")


# Agrégation par marché et année
grouped = df_filtered.groupby(["mar_diminutif", "com_bud_annee"])["com_prod_montant_ht"].sum().unstack()


# Moyenne des années 2023 et 2024
grouped["Montant Moy. 2023/24 (k€)"] = grouped[[2023, 2024]].mean(axis=1) / 1000


# Somme totale des moyennes pour tous les marchés
total_avg_all = grouped["Montant Moy. 2023/24 (k€)"].sum()


# Extraire les marchés sélectionnés et calculer les %
selected_avg = grouped.loc[grouped.index.isin(selected_marches), ["Montant Moy. 2023/24 (k€)"]].copy()
selected_avg["% du total moyen"] = (selected_avg["Montant Moy. 2023/24 (k€)"] / total_avg_all) * 100


# Arrondir à 2 chiffres significatifs
selected_avg = selected_avg.applymap(round_sf)


# Renommer la colonne et trier
selected_avg = selected_avg.reset_index().rename(columns={
    "mar_diminutif": "Marché"
})

selected_avg = selected_avg.sort_values("Montant Moy. 2023/24 (k€)", ascending=False)


# Affichage final
selected_avg


com_bud_annee,Marché,Montant Moy. 2023/24 (k€),% du total moyen
7,SignaDyn,660.0,8.300
3,CS,640.0,8.000
5,MDIRIF,120.0,1.500
0,AMO Tunnel,53.0,0.660
1,CAOA,28.0,0.350
6,MIISST2,18.0,0.230
2,CLIM2,13.0,0.160
4,Climatisation2,6.1,0.076
